# Hola &#x1F600;,

Soy **Hesus Garcia** – **"Soy el único Hesus que conoces (y probablemente conocerás) 🌟"** – Sí, como "Jesús", pero con una H que me hace único. Puede sonar raro, pero créeme, ¡no lo olvidarás! Como tu revisor en Triple-Ten, estoy aquí para guiarte y ayudarte a mejorar tu código. Si algo necesita un ajuste, no hay de qué preocuparse; ¡aquí estoy para hacer que tu trabajo brille con todo su potencial! ✨

Cada vez que encuentre un detalle importante en tu código, te lo señalaré para que puedas corregirlo y así te prepares para un ambiente de trabajo real, donde el líder de tu equipo actuaría de manera similar. Si en algún momento no logras solucionar el problema, te daré más detalles para ayudarte en nuestra próxima oportunidad de revisión.

Es importante que cuando encuentres un comentario, **no los muevas, no los modifiques, ni los borres**.

---

### Formato de Comentarios

Revisaré cuidadosamente cada implementación en tu notebook para asegurar que cumpla con los requisitos y te daré comentarios de acuerdo al siguiente formato:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>
    
<b>Éxito</b> - ¡Excelente trabajo! Esta parte está bien implementada y contribuye significativamente al análisis de datos o al proyecto. Continúa aplicando estas buenas prácticas en futuras secciones.
    
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>
    
<b>Atención</b> ⚠️ - Este código está correcto, pero se puede optimizar. Considera implementar mejoras para que sea más eficiente y fácil de leer. Esto fortalecerá la calidad de tu proyecto.
    
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>
    
<b>A resolver</b> ❗ - Aquí hay un problema o error en el código que es necesario corregir para aprobar esta sección. Por favor, revisa y corrige este punto, ya que es fundamental para la validez del análisis y la precisión de los resultados.
    
</div>

---

Al final de cada revisión, recibirás un **Comentario General del Revisor** que incluirá:

- **Aspectos positivos:** Un resumen de los puntos fuertes de tu proyecto.
- **Áreas de mejora:** Sugerencias sobre aspectos donde puedes mejorar.
- **Temas adicionales para investigar:** Ideas de temas opcionales que puedes explorar por tu cuenta para desarrollar aún más tus habilidades.

Estos temas adicionales no son obligatorios en esta etapa, pero pueden serte útiles para profundizar en el futuro.

---


Esta estructura en viñetas facilita la lectura y comprensión de cada parte del comentario final.

También puedes responderme de la siguiente manera si tienes alguna duda o quieres aclarar algo específico:


<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
    
Aquí puedes escribir tu respuesta o pregunta sobre el comentario.
    
</div>


**¡Empecemos!** &#x1F680;














# Proyecto Final — Predicción de Cancelación de Clientes (Interconnect)

El presente proyecto tiene como objetivo desarrollar un modelo predictivo que permita a la empresa Interconnect, un operador de telecomunicaciones, pronosticar la cancelación de sus clientes (churn).
Detectar con anticipación a los usuarios con mayor probabilidad de darse de baja permitirá ofrecerles promociones personalizadas y retenerlos, optimizando así la rentabilidad del negocio.

Para este propósito, se cuenta con información proveniente de distintas fuentes:

contract.csv: datos del contrato y estado del cliente.

personal.csv: características personales y demográficas.

internet.csv: servicios de internet contratados.

phone.csv: servicios telefónicos asociados.

A partir de estos datos, se realizará una integración, análisis exploratorio y modelado predictivo, con el fin de identificar los factores que influyen en la cancelación y construir un modelo capaz de estimar la probabilidad de churn de cada cliente.

## 1.- INICIALIZACION

In [1]:
import pandas as pd
from pathlib import Path

BASE = Path("/datasets/final_provider")

contract = pd.read_csv(BASE / "contract.csv")
personal = pd.read_csv(BASE / "personal.csv")
internet = pd.read_csv(BASE / "internet.csv")
phone    = pd.read_csv(BASE / "phone.csv")

contract["TotalCharges"] = pd.to_numeric(contract["TotalCharges"], errors="coerce")
contract["BeginDate"] = pd.to_datetime(contract["BeginDate"], errors="coerce")
contract["EndDate"]   = pd.to_datetime(contract["EndDate"], errors="coerce")
contract["churn"] = contract["EndDate"].notna().astype(int)

print(contract.info())
print()
print(contract.head())
print()
print(personal.info())
print()
print(personal.head())
print()
print(internet.info())
print()
print(internet.head())
print()
print(phone.info())
print()
print(phone.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7043 non-null   object        
 1   BeginDate         7043 non-null   datetime64[ns]
 2   EndDate           1869 non-null   datetime64[ns]
 3   Type              7043 non-null   object        
 4   PaperlessBilling  7043 non-null   object        
 5   PaymentMethod     7043 non-null   object        
 6   MonthlyCharges    7043 non-null   float64       
 7   TotalCharges      7032 non-null   float64       
 8   churn             7043 non-null   int64         
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 495.3+ KB
None

   customerID  BeginDate    EndDate            Type PaperlessBilling  \
0  7590-VHVEG 2020-01-01        NaT  Month-to-month              Yes   
1  5575-GNVDE 2017-04-01        NaT        One year               N

## 2.- RESUMEN INICIAL DE LOS DATOS

El archivo contract.csv contiene 7043 registros, con información del contrato y cargos mensuales.

La columna EndDate puede servir como variable objetivo (clientes activos vs cancelados).

TotalCharges parece ser texto (object) aunque representa valores numéricos, por lo que requerirá conversión.

El archivo personal.csv tiene 7043 registros con información demográfica (género, pareja, dependientes).

El archivo internet.csv parece contener registros a nivel de sesiones (149,396 filas), por lo que se deberá agrupar por cliente para unificarlo correctamente.

El archivo phone.csv contiene 6361 registros; algunos clientes podrían no tener servicio telefónico.

En general, las tablas se podrán unir mediante la columna customerID, siendo contract.csv la base principal.


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>

<b>Éxito</b> ✅ - Excelente trabajo en la exploración y carga de datos. Has integrado correctamente las diferentes fuentes y mostrado una comprensión sólida del contexto del proyecto. Además, tu análisis inicial es claro y útil para orientar el modelado.
Te felicito por esa estructura y te recomiendo incorporar el uso de <b>profilers</b> (`cProfile`, `line_profiler`, `memory_profiler`) para identificar posibles cuellos de botella en tus futuros experimentos.

</div>


## 3.-PREGUNTAS ACLARATORIAS

¿Cuál será exactamente la variable objetivo? ¿Se considera “cancelado” todo cliente con EndDate distinto de vacío?

¿Podemos asumir que todos los clientes activos al 1 de febrero de 2020 no se han dado de baja?

¿Existe algún criterio de balanceo para el dataset (si hay muy pocos clientes que cancelaron)?

¿Podemos usar técnicas de sobremuestreo como SMOTE o prefieren un modelo más simple?

¿Cuál será el criterio principal de éxito: precisión, recall o ROC-AUC?

En el archivo internet.csv, ¿debemos considerar la cantidad total de megabytes usados por cliente como una variable adicional (sumatoria por customerID), o solo validar si tiene servicio de Internet activo?


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>

<b>Atención</b> ⚠️ - Antes de continuar con el modelado, revisa lo siguiente:

* La variable objetivo es <b>churn</b>, derivada de la columna <code>EndDate</code> (1 si tiene fecha, 0 si no). Para esta variable, debes realizar <b>feature engineering</b> (duración del contrato, recencia, servicios activos, cargos, etc.).
* Puedes asumir ciertos comportamientos, pero <b>documenta siempre tus supuestos</b>.
* El <b>balanceo</b> de clases (por ejemplo, con <code>SMOTE</code> o <code>class_weight</code>) debe ser propuesto y justificado por ti, ya que el experto de negocio no es técnico.
* Al combinar métricas (ROC-AUC, Recall, PR-AUC), recuerda que <b>tú decides el punto de corte final</b> según tu criterio e intuición.
* Además, revisa los datasets desde la ruta correcta: <code>/datasets/final_provider/</code> para garantizar consistencia.

Ejemplo de lectura:

```python
import pandas as pd
from pathlib import Path

BASE = Path("/datasets/final_provider")

contract = pd.read_csv(BASE / "contract.csv")
personal = pd.read_csv(BASE / "personal.csv")
internet = pd.read_csv(BASE / "internet.csv")
phone    = pd.read_csv(BASE / "phone.csv")

contract["TotalCharges"] = pd.to_numeric(contract["TotalCharges"], errors="coerce")
contract["BeginDate"] = pd.to_datetime(contract["BeginDate"], errors="coerce")
contract["EndDate"]   = pd.to_datetime(contract["EndDate"], errors="coerce")
contract["churn"] = contract["EndDate"].notna().astype(int)
```

</div>

## 4.- Variable objetivo propuesta

La variable objetivo será churn, definida así:

churn = 1 → cliente cancelado (EndDate contiene fecha).

churn = 0 → cliente activo (EndDate vacío o “No”).

## 5.- Resumen de supuestos

Los datos corresponden a contratos válidos hasta el 1 de febrero de 2020.

Los registros con EndDate vacío representan clientes activos.

Los clientes sin servicio telefónico o internet son considerados válidos.

TotalCharges debe convertirse a numérico para análisis de gasto acumulado.

## 6.-PLAN DE TRABAJO

### 6.1 Preparación de datos
Unificar los cuatro datasets usando customerID.

Corregir tipos de datos (float, datetime, object).

Tratar valores nulos y registros duplicados.

Identificar posibles clientes sin servicios activos (caso phone o internet vacíos).

### 6.2 Feature Engineering

tenure_months: meses de relación del cliente con la empresa (EndDate - BeginDate).

recency_days: días desde el inicio del contrato hasta la fecha de corte.

charges_per_tenure: promedio de cargos mensuales.

active_services_count: cantidad de servicios activos (TV, soporte, backup, etc.).

is_fiber, is_dsl: indicadores binarios de tipo de conexión.

multiple_lines: indicador para clientes con varias líneas telefónicas.

Estas variables permitirán capturar la intensidad de uso y la fidelidad contractual.

### 6.3 Feature Selection

En la fase de modelado se evaluará la importancia de las variables mediante:

Correlación con la variable objetivo (análisis exploratorio inicial).

SelectKBest (Mutual Information) para priorizar variables relevantes.

Boruta (opcional): técnica basada en bosques aleatorios que refuerza la estabilidad de la selección.

Lasso (L1 regularization): selección automática de variables mediante penalización.

SHAP values (opcional): interpretabilidad de impacto individual por característica.

### 6.4 Modelado

Dividir el dataset en entrenamiento (75%) y prueba (25%).

Probar modelos base: Regresión Logística, Random Forest y Gradient Boosting.

Evaluar el desempeño con ROC-AUC y Recall.

Aplicar balanceo de clases (SMOTE, class_weight) según los resultados observados.

Ajustar umbral de decisión para optimizar recall sin sacrificar precisión.

### 6.5 Evaluación y conclusiones

Comparar métricas de los modelos y seleccionar el más robusto.

Analizar la importancia de características y su relación con la cancelación.

Proponer recomendaciones estratégicas para retención de clientes.

<s>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>

<b>A resolver</b> ❗ - El plan de trabajo necesita mayor <b>detalle, orden y estructura</b>.
Incluye encabezados en <code>Markdown</code> y agrega secciones específicas para <b>Feature Engineering</b> y <b>Feature Selection</b>.
Te recomiendo explorar técnicas más avanzadas como <b>Boruta</b> (basada en árboles), <b>Lasso</b> (para regularización y sparsity) y <b>SHAP</b> (para interpretabilidad).
Aunque no se vieron en el bootcamp, son una excelente oportunidad para fortalecer tu trabajo y mostrar iniciativa profesional.

</div>




<div class="alert alert-block alert-success">  
<b>Comentario del revisor</b> <a class="tocSkip"></a><br>  
<b>Éxito</b> - Qué buen progreso reflejas en esta segunda iteración, se nota el compromiso y la constancia con que has trabajado. Agradezco tu apertura para incorporar las sugerencias anteriores y mejorar la estructura del proyecto; el resultado es un avance notable en claridad y rigor técnico. Tu capacidad para adaptarte y aprender con rapidez marcará una gran diferencia en las siguientes etapas.  
</div>
